In [1]:
from keras.utils import to_categorical
import pandas as pd
import numpy as np
import cv2

def preprocessing(data, labels, val=True):
    
    data = pd.read_csv(data)
    labels = pd.read_csv(labels)

    if(val):
        ## ---------------- Data preparation ---------------- ##
        X_train = []
        for i in range(data.shape[0]):
            img = np.uint8(data.iloc[i])
            edited = cv2.Canny(img, 10, 30)
            edited = cv2.GaussianBlur(edited, (5, 5), 0)
            X_train.append(edited.reshape((1,-1))[0])

        data = pd.DataFrame(X_train)
        ## -------------------------------------------------- ##

    return data, labels

Using TensorFlow backend.


In [2]:
X_train, y_train = preprocessing(data='data/x_train_gr_smpl.csv', labels='data/y_train_smpl.csv')
X_train['label'] = y_train
X_train = X_train.sample(frac=1)
y_train = X_train['label']
X_train = X_train.iloc[:, :-1]
y_train = to_categorical(y_train)
X_train = np.array(X_train).reshape(-1, 48, 48, 1)
X_train = X_train/255.0

X_test, y_test = preprocessing(data='data/x_test_gr_smpl.csv', labels='data/y_test_smpl.csv')
X_test['label'] = y_test
X_test = X_test.sample(frac=1)
y_test = X_test['label']
X_test = X_test.iloc[:, :-1]
y_test = to_categorical(y_test)
X_test = np.array(X_test).reshape(-1, 48, 48, 1)
X_test = X_test/255.0

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time


gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

conv_layers = [1, 2, 3]
feature_maps = [32, 64]
dense_layers = [1, 2]
layer_sizes = [128, 256]

for conv_layer in conv_layers:
    for feature_map in feature_maps:
        for dense_layer in dense_layers:
            for layer_size in layer_sizes:

                NAME = f"{conv_layer}ConvL-{feature_map}FMaps-{dense_layer}DenseL-{layer_size}Neurons-{int(time.time())}"
                tb_callbacks = TensorBoard(log_dir=f"logs\\full\\{NAME}", write_graph=True)
                es_callbacks = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

                print("----------------------------------------")
                print(NAME)

                model = Sequential()

                model.add(Conv2D(feature_map, (5, 5), input_shape=X_train.shape[1:], activation='relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                for _ in range(conv_layer-1):
                    model.add(Conv2D(feature_map, (5, 5), activation='relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Flatten())
                for _ in range(dense_layer):
                    model.add(Dense(layer_size, activation='relu'))
                    model.add(Dropout(0.2))

                model.add(Dense(10, activation='softmax'))

                model.compile(loss="categorical_crossentropy",
                             optimizer="adam",
                             metrics=["accuracy"])

                model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=256, epochs=100, callbacks=[tb_callbacks, es_callbacks])
                model.save(f"models\\full\\{NAME}.model")

----------------------------------------
1ConvL-32FMaps-1DenseL-128Neurons-1575027731
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 12660 samples, validate on 4170 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
12660/12660 [==============================] - 3s 275us/sample - loss: 0.7079 - acc: 0.7707 - val_loss: 0.4363 - val_acc: 0.8158
Epoch 2/100
12660/12660 [==============================] - 2s 131us/sample - loss: 0.2342 - acc: 0.9147 - val_loss: 0.3252 - val_acc: 0.8914
Epoch 3/100
12660/12660 [==============================] - 2s 129us/sample - loss: 0.1593 - acc: 0.9457 - val_loss: 0.2862 - val_acc: 0.9072
Epoch 4/100
12660/12660 [==============================] - 2s 128us/sample - loss: 0.1184 - acc: 0.9595 - val_loss: 0.2828 - val_acc: 0.9031
Epoch 5/100
12660/12660 [==============================] - 2s 128u

12660/12660 [==============================] - 3s 244us/sample - loss: 0.0740 - acc: 0.9731 - val_loss: 0.2480 - val_acc: 0.9266
Epoch 7/100
12660/12660 [==============================] - 2s 186us/sample - loss: 0.0589 - acc: 0.9794 - val_loss: 0.2478 - val_acc: 0.9235
Epoch 8/100
12660/12660 [==============================] - 2s 151us/sample - loss: 0.0465 - acc: 0.9832 - val_loss: 0.2737 - val_acc: 0.9168
----------------------------------------
2ConvL-32FMaps-1DenseL-256Neurons-1575027865
Train on 12660 samples, validate on 4170 samples
Epoch 1/100
12660/12660 [==============================] - 3s 223us/sample - loss: 0.8552 - acc: 0.7140 - val_loss: 0.3799 - val_acc: 0.8691
Epoch 2/100
12660/12660 [==============================] - 2s 149us/sample - loss: 0.2298 - acc: 0.9161 - val_loss: 0.3182 - val_acc: 0.8890
Epoch 3/100
12660/12660 [==============================] - 2s 148us/sample - loss: 0.1368 - acc: 0.9494 - val_loss: 0.2967 - val_acc: 0.9062
Epoch 4/100
12660/12660 [======

12660/12660 [==============================] - 2s 151us/sample - loss: 0.1664 - acc: 0.9393 - val_loss: 0.2659 - val_acc: 0.9125
Epoch 5/100
12660/12660 [==============================] - 2s 156us/sample - loss: 0.1285 - acc: 0.9524 - val_loss: 0.2797 - val_acc: 0.9058
----------------------------------------
3ConvL-32FMaps-2DenseL-128Neurons-1575028046
Train on 12660 samples, validate on 4170 samples
Epoch 1/100
12660/12660 [==============================] - 3s 227us/sample - loss: 1.3296 - acc: 0.5408 - val_loss: 0.6015 - val_acc: 0.7683
Epoch 2/100
12660/12660 [==============================] - 2s 154us/sample - loss: 0.5018 - acc: 0.8112 - val_loss: 0.3804 - val_acc: 0.8664
Epoch 3/100
12660/12660 [==============================] - 2s 155us/sample - loss: 0.3305 - acc: 0.8703 - val_loss: 0.3560 - val_acc: 0.8825
Epoch 4/100
12660/12660 [==============================] - 2s 156us/sample - loss: 0.2305 - acc: 0.9122 - val_loss: 0.2938 - val_acc: 0.9007
Epoch 5/100
12660/12660 [======

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time


gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

conv_layers = [1, 2, 3]
feature_maps = [32, 64]
dense_layers = [1, 2]
layer_sizes = [128, 256]

for conv_layer in conv_layers:
    for feature_map in feature_maps:
        for dense_layer in dense_layers:
            for layer_size in layer_sizes:

                NAME = f"{conv_layer}ConvL-{feature_map}FMaps-{dense_layer}DenseL-{layer_size}Neurons-{int(time.time())}"
                tb_callbacks = TensorBoard(log_dir=f"logs\\split\\{NAME}", write_graph=True)
                es_callbacks = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

                print("----------------------------------------")
                print(NAME)

                model = Sequential()

                model.add(Conv2D(feature_map, (5, 5), input_shape=X_train.shape[1:], activation='relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                for _ in range(conv_layer-1):
                    model.add(Conv2D(feature_map, (5, 5), activation='relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Flatten())
                for _ in range(dense_layer):
                    model.add(Dense(layer_size, activation='relu'))
                    model.add(Dropout(0.2))

                model.add(Dense(10, activation='softmax'))

                model.compile(loss="categorical_crossentropy",
                             optimizer="adam",
                             metrics=["accuracy"])

                model.fit(X_train, y_train, validation_split=0.3, batch_size=256, epochs=100, callbacks=[tb_callbacks, es_callbacks])
                model.save(f"models\\split\\{NAME}.model")

----------------------------------------
1ConvL-32FMaps-1DenseL-128Neurons-1575028428
Train on 8862 samples, validate on 3798 samples
Epoch 1/100
8862/8862 [==============================] - 2s 241us/sample - loss: 0.9723 - acc: 0.6869 - val_loss: 0.4577 - val_acc: 0.8152
Epoch 2/100
8862/8862 [==============================] - 1s 140us/sample - loss: 0.3454 - acc: 0.8753 - val_loss: 0.2458 - val_acc: 0.9176
Epoch 3/100
8862/8862 [==============================] - 1s 138us/sample - loss: 0.2242 - acc: 0.9202 - val_loss: 0.1861 - val_acc: 0.9379
Epoch 4/100
8862/8862 [==============================] - 1s 141us/sample - loss: 0.1638 - acc: 0.9473 - val_loss: 0.1875 - val_acc: 0.9279
----------------------------------------
1ConvL-32FMaps-1DenseL-256Neurons-1575028445
Train on 8862 samples, validate on 3798 samples
Epoch 1/100
8862/8862 [==============================] - 2s 239us/sample - loss: 0.8689 - acc: 0.7079 - val_loss: 0.3346 - val_acc: 0.8752
Epoch 2/100
8862/8862 [==============

8862/8862 [==============================] - 2s 215us/sample - loss: 0.0420 - acc: 0.9874 - val_loss: 0.0752 - val_acc: 0.9734
Epoch 7/100
8862/8862 [==============================] - 2s 225us/sample - loss: 0.0256 - acc: 0.9931 - val_loss: 0.0749 - val_acc: 0.9745
Epoch 8/100
8862/8862 [==============================] - 2s 230us/sample - loss: 0.0168 - acc: 0.9959 - val_loss: 0.0641 - val_acc: 0.9795
Epoch 9/100
8862/8862 [==============================] - 2s 234us/sample - loss: 0.0143 - acc: 0.9966 - val_loss: 0.0737 - val_acc: 0.9742
----------------------------------------
2ConvL-32FMaps-1DenseL-128Neurons-1575028611
Train on 8862 samples, validate on 3798 samples
Epoch 1/100
8862/8862 [==============================] - 3s 304us/sample - loss: 1.1876 - acc: 0.6106 - val_loss: 0.4575 - val_acc: 0.8299
Epoch 2/100
8862/8862 [==============================] - 1s 160us/sample - loss: 0.3805 - acc: 0.8579 - val_loss: 0.2628 - val_acc: 0.8934
Epoch 3/100
8862/8862 [=====================

8862/8862 [==============================] - 2s 259us/sample - loss: 0.1465 - acc: 0.9455 - val_loss: 0.1213 - val_acc: 0.9573
Epoch 4/100
8862/8862 [==============================] - 2s 259us/sample - loss: 0.1070 - acc: 0.9605 - val_loss: 0.0974 - val_acc: 0.9629
Epoch 5/100
8862/8862 [==============================] - 2s 257us/sample - loss: 0.0831 - acc: 0.9665 - val_loss: 0.0970 - val_acc: 0.9605
Epoch 6/100
8862/8862 [==============================] - 2s 269us/sample - loss: 0.0571 - acc: 0.9806 - val_loss: 0.0689 - val_acc: 0.9734
Epoch 7/100
8862/8862 [==============================] - 2s 257us/sample - loss: 0.0426 - acc: 0.9852 - val_loss: 0.0771 - val_acc: 0.9697
----------------------------------------
2ConvL-64FMaps-2DenseL-128Neurons-1575028801
Train on 8862 samples, validate on 3798 samples
Epoch 1/100
8862/8862 [==============================] - 4s 421us/sample - loss: 1.1649 - acc: 0.6034 - val_loss: 0.4455 - val_acc: 0.8144
Epoch 2/100
8862/8862 [=====================

8862/8862 [==============================] - 2s 210us/sample - loss: 0.1579 - acc: 0.9421 - val_loss: 0.1367 - val_acc: 0.9516
Epoch 9/100
8862/8862 [==============================] - 2s 177us/sample - loss: 0.1386 - acc: 0.9491 - val_loss: 0.1195 - val_acc: 0.9566
Epoch 10/100
8862/8862 [==============================] - 2s 175us/sample - loss: 0.1197 - acc: 0.9564 - val_loss: 0.1113 - val_acc: 0.9605
Epoch 11/100
8862/8862 [==============================] - 2s 195us/sample - loss: 0.1085 - acc: 0.9622 - val_loss: 0.1030 - val_acc: 0.9647
Epoch 12/100
8862/8862 [==============================] - 2s 202us/sample - loss: 0.0861 - acc: 0.9680 - val_loss: 0.0947 - val_acc: 0.9658
Epoch 13/100
8862/8862 [==============================] - 2s 237us/sample - loss: 0.0856 - acc: 0.9685 - val_loss: 0.0917 - val_acc: 0.9689
Epoch 14/100
8862/8862 [==============================] - 2s 184us/sample - loss: 0.0781 - acc: 0.9726 - val_loss: 0.0956 - val_acc: 0.9666
----------------------------------

tensorboard --logdir=logs/full --host localhost --port 8088

tensorboard --logdir=logs/split --host localhost --port 8088